#### Spacy test

In [2]:
import spacy
nlp_de = spacy.load("de_core_news_lg")

In [3]:
import pandas as pd
train_text = "Sie studierte von 1902 bis 1907 an der Universität Königsberg Medizin und wurde 1909 promoviert. Anschließend arbeitete er in einem Krankenhaus und machte 1918 das Kreisarztexamen.[1] 1922[2] beschrieben Hallervorden und der Berliner Neuropathologe Hugo Spatz erstmals eine Krankheit, die zunächst Hallervorden-Spatz-Syndrom (HSS) genannt wurde[3]. Dieses Eponym wird heute aufgrund der nationalsozialistischen Vergangenheit von Hallervorden und Spatz (siehe weiter unten) zunehmend nicht mehr verwendet. Die Erkrankung wird heute je nach Kontext und Symptomen Pantothenatkinase-assoziierte Neurodegeneration oder allgemeiner Neurodegeneration mit Eisenablagerung im Gehirn genannt.[4][5]"
doc = nlp_de(train_text)
pd.DataFrame({"Token": [token.text for token in doc],
              "Lemma": [token.lemma_ for token in doc],
              "POS": [token.pos_ for token in doc],
              "Tag": [token.tag_ for token in doc],
              "Morph": [token.morph for token in doc],
              "Dep": [token.dep_ for token in doc]})



,Token,Lemma,POS,Tag,Morph,Dep
0,Sie,ich,PRON,PPER,"(Case=Nom, Gender=Fem, Number=Sing, Person=3, ...",sb
1,studierte,studieren,VERB,VVFIN,"(Mood=Ind, Number=Sing, Person=3, Tense=Past, ...",ROOT
2,von,von,ADP,APPR,(),mo
3,1902,1902,NUM,CARD,(),nk
4,bis,bis,ADP,APPR,(),mo
...,...,...,...,...,...,...
89,Eisenablagerung,Eisenablagerung,NOUN,NN,"(Case=Dat, Gender=Fem, Number=Sing)",nk
90,im,im,ADP,APPRART,"(Case=Dat, Gender=Neut, Number=Sing)",mnr
91,Gehirn,Gehirn,NOUN,NN,"(Case=Dat, Gender=Neut, Number=Sing)",nk
92,genannt.[4][5,genannt.[4][5,NOUN,NN,"(Case=Nom, Gender=Masc, Number=Sing)",ROOT


In [4]:
suffixes = ["er", "ist", "ant", "ent", "eur", "ar"]
noun_list = []
print('---nouns---')
for token in doc:
    if len(token.morph.get('Gender')) >= 1 and token.morph.get('Gender')[0] == "Masc" and (token.pos_ == 'NOUN'):
      
        print(token.text + ', ' + token.tag_ +
              ', ' + token.dep_ + ', ' + token.pos_ + ', ' + token.lemma_ + ', ' + str(token.idx) + ', ' + str(token.idx + len(token.text)))
            
        noun_list.append(token)

adjectives = []

print('---adjectives---')
for token in noun_list:
    for child in token.children:
        if child.pos_ == "ADJ":
            adjectives.append(child)

for token in adjectives:
    print(token.text)

---nouns---
Neuropathologe, NN, cj, NOUN, Neuropathologe, 234, 248
Spatz, NN, cj, NOUN, Spatz, 446, 451
Kontext, NN, nk, NOUN, Kontext, 539, 546
genannt.[4][5, NN, ROOT, NOUN, genannt.[4][5, 674, 687
---adjectives---
Berliner


In [5]:
print(noun_list)

toDelete = []
#toDelete.append(2)

def clearMasculine(list):
    for i in toDelete:
        del noun_list[i]
    return

clearMasculine(toDelete)
print(noun_list)

[Neuropathologe, Spatz, Kontext, genannt.[4][5]
[Neuropathologe, Spatz, Kontext, genannt.[4][5]


In [6]:
class EditedWord:
    def __init__(self, text, pos1, pos2):
        self.text = text
        self.pos1 = pos1
        self.pos2 = pos2
    def print(self):
        print(self.text + "; " + str(self.pos1) + ":" +str(self.pos2))

edited_nouns = []
edited_nouns.clear()
print(noun_list)

for token in noun_list:
    if len(token.morph.get('Number')) >= 1 and token.morph.get('Number') == "Plur":
        newtext = token.text + "innen"
    elif len(token.morph.get('Number')) >= 1:
        newtext = token.text + "in"

    #print (newtext)
   
    edited_nouns.append(EditedWord(newtext, token.idx, (token.idx + len(token.text))))

for obj in edited_nouns:
     obj.print()

[Neuropathologe, Spatz, Kontext, genannt.[4][5]
Neuropathologein; 234:248
Spatzin; 446:451
Kontextin; 539:546
genannt.[4][5in; 674:687


In [10]:
print(spacy.explain("sb"))

subject


In [8]:

def replace_words (original_text, words):
    text=""
    i = 0
    for word in words:
        
        word.print()
        left, right = original_text[:word.pos1+i], original_text[word.pos2+i:]
        text = left + word.text + right
        print(text)
        original_text = text
        i+=2
   

replace_words(train_text, edited_nouns)

Neuropathologein; 234:248
Sie studierte von 1902 bis 1907 an der Universität Königsberg Medizin und wurde 1909 promoviert. Anschließend arbeitete er in einem Krankenhaus und machte 1918 das Kreisarztexamen.[1] 1922[2] beschrieben Hallervorden und der Berliner Neuropathologein Hugo Spatz erstmals eine Krankheit, die zunächst Hallervorden-Spatz-Syndrom (HSS) genannt wurde[3]. Dieses Eponym wird heute aufgrund der nationalsozialistischen Vergangenheit von Hallervorden und Spatz (siehe weiter unten) zunehmend nicht mehr verwendet. Die Erkrankung wird heute je nach Kontext und Symptomen Pantothenatkinase-assoziierte Neurodegeneration oder allgemeiner Neurodegeneration mit Eisenablagerung im Gehirn genannt.[4][5]
Spatzin; 446:451
Sie studierte von 1902 bis 1907 an der Universität Königsberg Medizin und wurde 1909 promoviert. Anschließend arbeitete er in einem Krankenhaus und machte 1918 das Kreisarztexamen.[1] 1922[2] beschrieben Hallervorden und der Berliner Neuropathologein Hugo Spatz erst